[![cloudevel](img/cloudevel.png)](https://cloudevel.com)

# El archivo ```Dockerfile```

*Docker* nos permite crear nuestras propias imágenes contenedores mediante el uso de un archivo de configuración llamado ``Dockerfile``. Este archivo es un *script*, el cual permite crear una imagen de contenedor a partir de otra y realizar diversas operaciones con este.

* https://docs.docker.com/engine/reference/builder/

## Comandos de ```Dockerfile```

* ```FROM``` Es el comando que importa la imagen base a partir de la cual se creará la nueva imagen.
* ```USER``` Define la UID del usuario que será utilizado.
* ```VOLUME``` Liga a un directorio local con el contenedor.
* ```RUN``` La cual ejecuta diversos comandos al momento de la creación del contenedor.
* ```COPY``` Copia un archivo del sistema de archivos local al sistema de archivos de la imagen.
* ```ENV``` Define variables de entorno.
* ```CMD``` Ejecuta comandos al momento de instanciar un contenedor a partir de la imagen.
* ```ENTRYPOINT``` Define el comando que se utilizará por defecto al ejecutar un contenedor instanciado de la imagen.
* ```WORKDIR``` Define el directorio de trabajo dentro del contenedor.
* ```EXPOSE``` define el reenvío de puertos.

## Construcción de una imagen a partir de un archivo ```Dockerfile```.

Para crear una imagen se utiliza el comando ```docker build``` de la siguiente forma:

```docker build -t <nombre de la imagen> <directorio de la Dockerfile>```

* https://docs.docker.com/engine/reference/commandline/build/

## Ejemplo ilustrativo.

A continuación se muestra un archivo ```Dockerfie``` el cual define los paso necesarios para construir un contendor que incluye un servicio basado en [*Nginx*](https://nginx.org/) el cual es gestionado mediante el daemon [*supervisord*](http://supervisord.org/). Además de dicho archivo se msotrarán los archivos:

* ```default```, el cual contiene la configuración genérica de un servidor *Nnginx*.
* ```supervisord```, el cual contiene la configuración de *supervisord*.

### El archivo ```Dockerfile```.

* El archivo localizado en [```src/04/Dockerfile```](src4/Dockerfile) define lo siguiente:

* Se usará la imagen ```ubuntu:20.04``` como la base de la nueva imagen.
* Se actualizarán los paquetes del sistema.
* Se instalarán los paquetes relacionados con *Nginx*, *supervisrod* y *PHP*.
* Se definirán las variables de entorno:
    * ```nginx_vhost``` apuntando al archivo de configuración del servicio basado en *Nginx* localizado en ```/etc/nginx/sites-available/default``` dentro del contenedor.
    *  ```php_conf``` apuntando al archivo de configuraciónde *PHP* localizado en ```/etc/php/7.4/fpm/php.ini```.
    * ```nginx_conf``` apuntando al archivo de configuración general de *Nginx* que apunta  a ```/etc/nginx/nginx.conf``` dentro del contenedor.
    * ```supervisor_conf``` apuntando al archivo de configuración de *supervisord* localizado en ```/etc/supervisor/supervisord.conf``` dentro del contenedor.
* Se copiará el archivo ```default``` localizado en el mismo directorio donde se encuentra el archivo ```Dockerfile```  en la localización indicada por la variable de entorno ```nginx_vhost```.
* Se añadirán algunos parámetros al archivo al que apunta la variable de entorno ```nginx_conf```.
* Se copiará el archivo ```supervisord.conf``` localizado en el mismo directorio donde se encuentra el archivo ```Dockerfile``` en la localización indicada por la variable de entorno ```supervisor_conf```.
* Se creará el directorio ```/run/php``` dentro del contenedor.
* Se definirá la propiedad de los directorios ```var/www/html``` y ```/run/php``` a favor del usuario y el grupo al que pertenece el servidor web.
* Expone los siguientes directorios para que puedan ser ligador a un directorio externo mediante la opción ```--volume``` de ```docker run```. 
    * ```/etc/nginx/sites-enabled```
    * ```/etc/nginx/certs```
    * ```/etc/nginx/conf.d```
    * ```/var/log/nginx```
    * ```/var/www/html```
* Copia al archivo ```script.sh``` localizado en el mismo directorio donde se encuentra el archivo ```Dockerfile```  en la raíz del sistema.
* Define al script ```/start.sh``` como el comando que se ejecutará al instanciar la imagen resultante.
* Expone los puertos ```80```y ```443``` para que puedan ser reenviados mediante la opción ```-ports```
 de ```docker run```.
 
``` bash
#Descarga la imagen
FROM ubuntu:20.04

# Actualiza el sistema
RUN apt-get update

# Instala nginx, php-fpm y supervisord
RUN apt-get install -y nginx php7.4 php7.4-fpm supervisor && \
    rm -rf /var/lib/apt/lists/*

#Define las variables de entorno
ENV nginx_vhost /etc/nginx/sites-available/default
ENV php_conf /etc/php/7.4/fpm/php.ini
ENV nginx_conf /etc/nginx/nginx.conf
ENV supervisor_conf /etc/supervisor/supervisord.conf

# habilita la configuración del servidor virtual de php-fpm en nginx
COPY default ${nginx_vhost}
RUN sed -i -e 's/;cgi.fix_pathinfo=1/cgi.fix_pathinfo=0/g' ${php_conf} && \
    echo "\ndaemon off;" >> ${nginx_conf}

#Copia la configuración del supervisor
COPY supervisord.conf ${supervisor_conf}

RUN mkdir -p /run/php && \
    chown -R www-data:www-data /var/www/html && \
    chown -R www-data:www-data /run/php

# Configura el volumen de almacenamiento
VOLUME ["/etc/nginx/sites-enabled", "/etc/nginx/certs", "/etc/nginx/conf.d", "/var/log/nginx", "/var/www/html"]

# Configura los servicios y los puertos 
COPY start.sh /start.sh
CMD ["./start.sh"]

EXPOSE 80 443
```

### El archivo ```default```.

El archivo [src/04/default]([src/04/default) contiene la configuración genérica de un servicio basado en *Nginx*. 

Cabe hacer notar que el directorio raíz del servicio es ```/var/www/html```.

``` json
server {
    listen 80 default_server;
    listen [::]:80 default_server;
 
    root /var/www/html;
    index index.html index.htm index.nginx-debian.html;
 
    server_name _;
 
    location / {
        try_files $uri $uri/ =404;
    }
 
    location ~ \.php$ {
        include snippets/fastcgi-php.conf;
        fastcgi_pass unix:/run/php/php7.4-fpm.sock;
    }
 
    # deny access to .htaccess files, if Apache's document root
    # concurs with nginx's one
    #
    #location ~ /\.ht {
    #    deny all;
    #}
}
```

### El archivo ```supervisord```.

* El archivo [src/4/supervisord](src/4/supervisord) contiene la configuración del daemon ```supervisor```.

```
[unix_http_server]
file=/dev/shm/supervisor.sock   ; (the path to the socket file)
 
[supervisord]
logfile=/var/log/supervisord.log ; (main log file;default $CWD/supervisord.log)
logfile_maxbytes=50MB        ; (max main logfile bytes b4 rotation;default 50MB)
logfile_backups=10           ; (num of main logfile rotation backups;default 10)
loglevel=info                ; (log level;default info; others: debug,warn,trace)
pidfile=/tmp/supervisord.pid ; (supervisord pidfile;default supervisord.pid)
nodaemon=false               ; (start in foreground if true;default false)
minfds=1024                  ; (min. avail startup file descriptors;default 1024)
minprocs=200                 ; (min. avail process descriptors;default 200)
user=root             ;
 
; the below section must remain in the config file for RPC
; (supervisorctl/web interface) to work, additional interfaces may be
; added by defining them in separate rpcinterface: sections
[rpcinterface:supervisor]
supervisor.rpcinterface_factory = supervisor.rpcinterface:make_main_rpcinterface
 
[supervisorctl]
serverurl=unix:///dev/shm/supervisor.sock ; use a unix:// URL  for a unix socket
 
; The [include] section can just contain the "files" setting.  This
; setting can list multiple files (separated by whitespace or
; newlines).  It can also contain wildcards.  The filenames are
; interpreted as relative to this file.  Included files *cannot*
; include files themselves.
 
[include]
files = /etc/supervisor/conf.d/*.conf
 
 
[program:php-fpm7.0]
command=/usr/sbin/php-fpm7.4 -F
numprocs=1
autostart=true
autorestart=true
 
[program:nginx]
command=/usr/sbin/nginx
numprocs=1
autostart=true
autorestart=true
```

### El script ```start.sh```.

* El archivo [src/4/start.sh](src/4/supervisord) tiene permisos de ejecución y contiene lo siguiente:

```
#!/bin/sh
 
/usr/bin/supervisord -n -c /etc/supervisor/supervisord.conf
```

### Creación de una imagen a partir del archivo ```Dockerfile```.

* El shell de esta notebook se moverá al sudirectorio ```src```.

In [ ]:
%cd src/04

* Se ejecutará el comando ```docker build``` para crear la imagen con nombre ```nginx_image``` a partir del archivo ```Dockerfile``` que se encuentra en el directorio actual.

In [ ]:
!ls Dockerfile

In [ ]:
!docker build -t nginx_image .

In [ ]:
!docker images

### Creación de un contenedor a partir de la nueva imagen.

* Se creará el subdirectorio ```webroot```.

In [ ]:
!mkdir webroot

* Se detendrá al servidor *Apache* que se encuentra corriendo en la máquina virtual porporcionada por Cloudevel®.

In [ ]:
!sudo systemctl stop apache2
!sudo systemctl status apache2 --no-pager

* Se creará un contenedor instanciado de ```nginx_image``` con las siguientes características:
    * Ligará al subdirectorio ```webroot``` con el directorio ```/var/www/html``` del contenedor.
    * Definirá un reenvío de puertos del puerto ```80``` del contenedor al puerto ```80``` de la máquina virtual.
    * El nuevo contenedor tendrá el nombre ```servidor```.

In [ ]:
!docker run -d -v $(pwd)/webroot:/var/www/html -p 80:80 --name servidor nginx_image

* Si se está ejecutando  esta notebook direactamente desde su equipo, el resultado de la ejecución de este contenedor puede ser vista desde http://localhost
* Si se está ejecutando  esta notebook desde la VM provista del sitio de Cloudevel®, el resultado de la ejecución de este contenedor puede ser vista desde http://localhost:8980
* En ambos casos, se verá que existe un servicio web basado en *NGINX* con el mensaje de error ```403 Forbidden```.

* Se copiará el archivo ```index.html``` al subdirectorio ```webroot```, de tal forma que su contenido sea desplegado por el servidor web.

In [ ]:
%cp index.html webroot/

*  Se detendrá y eliminará al contenedor ```servidor```.

In [ ]:
!docker stop servidor

In [ ]:
!docker rm servidor

* La imagen ```nginx_image``` continúa en el sistema.

In [ ]:
!docker images

* Se eliminará el subdirectorio ```webroot```.

In [ ]:
!rm -rf webroot

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2023.</p>